In [ ]:
import numpy as np
import pickle # lưu trữ và tải dữ liệu dạng nhị phân

In [ ]:
def unpickle(file):  # Mở đọc file dạng nhị phân --> trả về dữ liệu đã giải mã
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='bytes')
    return d

def load_cifar(filenames):
    training_images = []
    training_labels = []

    for file_name in filenames:
        unpickled_images = unpickle(file_name)
        images, labels = unpickled_images[b'data'], unpickled_images[b'labels']  # Lấy ảnh và nhãn,
        #'b' --> do khi đọc dữ liệu dạng nhị phân, dữ liệu được lưu dạng byte, vì vậy các khóa có dạng byte string
                                                     # Dữ liệu gốc (số lượng, 32x32x3) với số cột 1024: R, 1024:G, 1024:B
        images = np.reshape(images,(-1, 3, 32, 32))  # (number_of_images, channels (RGB), height, width)
        images = np.transpose(images, (0, 2, 3, 1))  # (number_of_images, height, width, channels (RGB)) --> phù hợp với hầu hết các thư viện xử lí ảnh và học sâu
        training_images.append(images)
        training_labels += labels

    return np.vstack(training_images), training_labels  # xếp chồng các theo trục đầu tiên --> trả về mảng chứa tất cả các ảnh

print("Loading the training set")
training_files = [f'/kaggle/input/chap-6/data_batch_{i}' for i in range(1, 6)]
training_images, int_training_labels = load_cifar(training_files)

print("Loading the testing set")
training_files = ['/kaggle/input/chap-6/test_batch']
testing_images, int_testing_labels = load_cifar(training_files)

print("Loading the labels")
label_names = unpickle('/kaggle/input/chap-6/batches.meta')[b'label_names']
training_labels = [str(label_names[i]) for i in int_training_labels]
testing_labels = [str(label_names[i]) for i in int_testing_labels]


In [ ]:
', '.join([l.decode() for l in label_names]) # kết hợp các nhãn thành một chuỗi duy nhất, mỗi nhãn cách nhau bởi dấu phẩy và khoảng trắng.

In [ ]:
import matplotlib.pyplot as plt # hiển thị một hình ảnh từ tập dữ liệu huấn luyện 
print(training_labels[0])
plt.imshow(training_images[0])
plt.title(training_labels[0])

In [ ]:
print(testing_labels[0]) # hiển thị một hình ảnh từ tập dữ liệu kiểm tra
plt.imshow(testing_images[0])

In [ ]:
training_images.shape

In [ ]:
training_images[0][0, 0]  # Truy cập 1 pixel cụ thể của 1 ảnh

## 1. Sử dụng trực tiếp pixel làm đặc trưng

In [ ]:
avg_training_images = training_images.mean(axis=3).reshape(50000,-1) # chuyển đổi ảnh màu trong tập huấn luyện và kiểm tra thành ảnh xám
avg_testing_images = testing_images.mean(axis=3).reshape(10000,-1)
# (50000, 32, 32, 3) -->(50000, 32, 32) --> (50000, 32*32) = (50000, 1024) # sau đó "trải phẳng" từng ảnh xám thành một mảng một chiều 1024 phần tử
print(avg_training_images.shape)

In [ ]:
plt.imshow(training_images.mean(axis=3)[0], cmap='gray') # hiển thị ảnh đầu tiên trong tập dữ dưới dạng ảnh xám 

In [ ]:
%run /kaggle/input/chap-6/Base.ipynb

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=100, solver='saga')  # solver: xác định bộ giải được sử dụng để tối ưu hóa trọng số của mô hình.

ml_pipeline = Pipeline([
    ('classifier', # tên bước
     clf  # đối tượng mô hình
     )
])

params = {
    'classifier__C': [1e-1, 1e0, 1e1] # chỉ định giá trị tham số C --> Tham số C điều chỉnh mức độ regularization trong mô hình
}

print("Average Pixel Value + LogReg\n==========================")
advanced_grid_search(   # D
    avg_training_images, training_labels, avg_testing_images, testing_labels,
    ml_pipeline, params
)


## Trích xuất đặc trưng:HOG(Histogram of oriented gradients)


In [ ]:
# tính toán và hiển thị tính năng Histogram of Oriented Gradients (HOG) của các hình ảnh trong training_images
from skimage.feature import hog 

for image in training_images[:3]:

    hog_features, hog_image = hog(
        image,
        orientations=8,
        pixels_per_cell=(4, 4),
        cells_per_block=(2, 2),
        visualize=True,
        channel_axis=-1,
        transform_sqrt=True,
        block_norm='L2-Hys'
        )

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(image, cmap=plt.cm.gray)
    ax1.set_title('Input image')

    ax2.axis('off')
    ax2.imshow(hog_image, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')

print(hog_features.shape)
plt.show()


In [ ]:
from tqdm import tqdm
#tqdm sẽ cung cấp cho chúng ta một thanh tiến trình để chúng ta có thể thấy mất bao lâu để tính toán HOGs.

def calculate_hogs(images):
    hog_descriptors = []
    for image in tqdm(np.sqrt(images)):
        hog_descriptors.append(hog(
            image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(2, 2),
            channel_axis=-1, transform_sqrt=True, block_norm='L2-Hys', visualize=False
        ))

    return np.squeeze(hog_descriptors)

hog_training = calculate_hogs(training_images)
hog_testing = calculate_hogs(testing_images)

In [ ]:
hog_training.shape

In [ ]:
hog_testing.shape

In [ ]:
print("HOG + LogReg\n=====================")
advanced_grid_search(
    hog_training, training_labels, hog_testing, testing_labels,
    ml_pipeline, params
)

### Tối ưu hóa giảm chiều bằng PCA

In [ ]:
from sklearn.decomposition import PCA

num_hog_features = hog_training.shape[1]  # B

p = PCA(n_components=num_hog_features)  # C
p.fit(hog_training)  # C

plt.plot(p.explained_variance_ratio_.cumsum())  # D
plt.title('Explained Variance vs # of PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('% of Explained Variance')

# A Nhập mô-đun PCA từ thư viện scikit-learn để thực hiện giảm chiều
# B Số lượng đặc trưng từ phép biến đổi HOG ban đầu
# C Khớp mô hình PCA với ma trận HOG.
# D Trực quan hóa xác định số lượng thành phần chính cần thiết cho việc giảm chiều.

In [ ]:
#  đánh giá xem việc giữ lại bao nhiêu thành phần chính là hợp lý để giải thích được phần lớn biến động của dữ liệu.
explained_variance = p.explained_variance_ratio_.cumsum()

for i in [10, 100, 200, 400]:
    print(f'The explained variance using {i} components is {explained_variance[i - 1]}')

In [ ]:
p = PCA(n_components=600)  # A

hog_training_pca = p.fit_transform(hog_training)  # B
hog_testing_pca = p.transform(hog_testing)  # B

print("HOG + PCA + LogReg\n=====================")
advanced_grid_search(  # C
    hog_training_pca, training_labels, hog_testing_pca, testing_labels,
    ml_pipeline, params
)

# A Chọn để trích xuất 600 đặc trưng mới.
# B Chuyển đổi các đặc trưng HOG gốc thành không gian giảm chiều
# C Lấy độ chính xác cho các đặc trưng HOG đã giảm chiều.

## Feature learning with VGG-11

In [ ]:
import torchvision.models as models
import torch.nn as nn

vgg_model = models.vgg11(pretrained='imagenet')  # A

normalized_training_images = ((training_images/255) - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]  # B
normalized_testing_images = ((testing_images/255) - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]  # B

# A Khởi tạo một mô hình VGG-11 đã được huấn luyện trước trên bộ dữ liệu ImageNet
# B Chuẩn hóa các hình ảnh thô sử dụng các giá trị từ nghiên cứu gốc

In [ ]:
vgg_model

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

training_images_tensor = torch.Tensor(normalized_training_images.transpose(0, 3, 1, 2))  # A
training_labels_tensor = torch.Tensor(int_training_labels).type(torch.LongTensor)  # A

training_dataset = TensorDataset(training_images_tensor, training_labels_tensor)  # A
training_dataloader = DataLoader(training_dataset, shuffle=True, batch_size=2048)  # A


testing_images_tensor = torch.Tensor(normalized_testing_images.transpose(0, 3, 1, 2))  # B
testing_labels_tensor = torch.Tensor(int_testing_labels).type(torch.LongTensor)  # B

testing_dataset = TensorDataset(testing_images_tensor, testing_labels_tensor)  # B
testing_dataloader = DataLoader(testing_dataset, shuffle=True, batch_size=2048)  # B

# A Chuyển đổi dữ liệu huấn luyện chuẩn hóa thành PyTorch DataLoader
# B Chuyển đổi dữ liệu kiểm tra chuẩn hóa thành PyTorch DataLoader

In [ ]:
from tqdm import tqdm # thư viện giúp hiển thị thanh tiến trình

def get_vgg_features(feature_extractor): # mô hình này sẽ được sử dụng để trích xuất các đặc trưng từ hình ảnh.
    print("Extracting features for training set") # việc trích xuất đặc trưng cho tập huấn luyện đang được thực hiện.
    extracted_training_images = [] # tạo danh sách để chứa các đặc trưng và nhãn của dữ liệu huấn luyện.
    shuffled_training_labels = []
    for batch_idx, (data_, target_) in tqdm(enumerate(training_dataloader)): # A
        extracted_training_images.append(feature_extractor(data_).detach().numpy().squeeze((2, 3))) # A
        shuffled_training_labels += target_ # A

    print("Extracting features for testing set") # việc trích xuất đặc trưng cho tập kiểm tra đang được thực hiện.
    extracted_testing_images = [] # tạo danh sách để chứa các đặc trưng và nhãn của dữ liệu kiểm tra.
    shuffled_testing_labels = []
    for batch_idx, (data_, target_) in tqdm(enumerate(testing_dataloader)): # B
        extracted_testing_images.append(feature_extractor(data_).detach().numpy().squeeze((2, 3))) # B
        shuffled_testing_labels += target_ # B

    return np.vstack(extracted_training_images), shuffled_training_labels, np.vstack(extracted_testing_images), shuffled_testing_labels
#A Dữ liệu và nhãn của tập huấn luyện được lấy từ training_dataloader và các đặc trưng được trích xuất bằng cách sử dụng feature_extractor.
#B Dữ liệu và nhãn của tập kiểm tra được lấy từ testing_dataloader và các đặc trưng được trích xuất bằng cách sử dụng feature_extractor.

In [ ]:
transformed_training_images, shuffled_training_labels, transformed_testing_images, shuffled_testing_labels = get_vgg_features(vgg_model.features)  # A

print("VGG11(Imagenet) + LogReg\n=====================")
advanced_grid_search(
    transformed_training_images, shuffled_training_labels, # B
    transformed_testing_images, shuffled_testing_labels,   # B
    ml_pipeline, params
)

# A trích xuất các đặc trưng từ mô hình VGG11 đã được huấn luyện trước trên Imagenet.
# B Việc gọi lại nhãn từ shuffled_training_labels và shuffled_testing_labels là cần thiết vì trong quá trình lấy dữ liệu từ DataLoader, các nhãn có thể đã bị xáo trộn.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  # A

fine_tuned_vgg_model = models.vgg11(pretrained='imagenet')  # B

fine_tuned_vgg_model.classifier[-1].out_features = 10  # C

for layer in fine_tuned_vgg_model.classifier:  # D
    if hasattr(layer, 'weight'):  # D
        torch.nn.init.xavier_uniform_(layer.weight)  # D
    if hasattr(layer, 'bias'):  # D
        nn.init.constant_(layer.bias.data, 0)  # D

# A Kiểm tra và thiết lập thiết bị tính toán (GPU hoặc CPU).
# B Tải mô hình VGG-11 mới
# C Thay đổi số lớp đầu ra của mô hình thành 10 lớp thay cho 1,000 (cho tác vụ phân loại với 10 lớp).
# D Duyệt qua các lớp trong phần classifier và khởi tạo lại trọng số và bias của chúng, giúp tinh chỉnh mô hình bắt đầu từ trạng thái ngẫu nhiên thay vì giá trị mặc định.

In [ ]:
import torch.optim as optim  # A

criterion = nn.CrossEntropyLoss()  # A
optimizer = optim.SGD(fine_tuned_vgg_model.parameters(), lr=0.01, momentum=0.9)  # A

n_epochs = 15  # A
print_every = 10  # A
valid_loss_min = np.Inf  # A
total_step = len(training_dataloader)  # A

train_loss, val_loss, train_acc, val_acc = [], [], [], []  # B


# A Đặt các tham số huấn luyện (điều chỉnh tham số chạy ngầm off-screeen)
# B Khởi tạo danh sách để theo dõi mất mát và độ chính xác

In [ ]:
# code inspired from https://www.pluralsight.com/guides/introduction-to-resnet
    
    # Huấn luyện mô hình qua nhiều epochs
for epoch in range(1, n_epochs + 1):
    running_loss = 0.0  # Tổng mất mát trong một epoch
    correct = 0  # Số lượng dự đoán đúng
    total = 0  # Tổng số mẫu trong epoch
    print(f'Epoch {epoch}\n')

    # Huấn luyện qua các mini-batches trong tập huấn luyện
    for batch_idx, (data_, target_) in enumerate(training_dataloader):
        data_, target_ = data_.to(device), target_.to(device)  # Di chuyển dữ liệu vào thiết bị (CPU hoặc GPU)
        optimizer.zero_grad()  # Xoá gradient trước khi tính toán gradient mới

        outputs = fine_tuned_vgg_model(data_)  # Dự đoán đầu ra từ mô hình
        loss = criterion(outputs, target_)  # Tính toán mất mát
        loss.backward()  # Lan truyền gradient ngược
        optimizer.step()  # Cập nhật trọng số của mô hình

        running_loss += loss.item()  # Cộng dồn mất mát
        _, pred = torch.max(outputs, dim=1)  # Tính toán dự đoán
        correct += torch.sum(pred == target_).item()  # Tính số lượng dự đoán đúng
        total += target_.size(0)  # Tổng số mẫu trong batch

        # In kết quả sau mỗi vài bước huấn luyện
        if (batch_idx) % print_every == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    # Tính toán và lưu trữ độ chính xác và mất mát trung bình trên tập huấn luyện
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss / total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct / total):.4f}%')

    batch_loss = 0  # Mất mát trên tập kiểm tra
    total_t = 0  # Tổng số mẫu trong tập kiểm tra
    correct_t = 0  # Số lượng dự đoán đúng trên tập kiểm tra

    # Đánh giá mô hình trên tập kiểm tra (validation)
    with torch.no_grad():  # Tắt gradient khi đánh giá
        fine_tuned_vgg_model.eval()  # Đặt mô hình vào chế độ đánh giá
        for data_t, target_t in (testing_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)  # Di chuyển dữ liệu vào thiết bị
            outputs_t = fine_tuned_vgg_model(data_t)  # Dự đoán đầu ra từ mô hình
            loss_t = criterion(outputs_t, target_t)  # Tính toán mất mát trên tập kiểm tra
            batch_loss += loss_t.item()  # Cộng dồn mất mát trên tập kiểm tra
            _, pred_t = torch.max(outputs_t, dim=1)  # Tính toán dự đoán trên tập kiểm tra
            correct_t += torch.sum(pred_t == target_t).item()  # Tính số lượng dự đoán đúng trên tập kiểm tra
            total_t += target_t.size(0)  # Tổng số mẫu trong tập kiểm tra

        # Tính toán và lưu trữ độ chính xác và mất mát trên tập kiểm tra
        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss / len(testing_dataloader))
        network_learned = batch_loss < valid_loss_min  # Kiểm tra xem mô hình có cải thiện không

        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}%\n')

        # Lưu mô hình nếu có sự cải thiện về mất mát
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(fine_tuned_vgg_model.state_dict(), 'vgg_cifar10.pt')  # Lưu trọng số mô hình
            print('Saving Parameters')

    fine_tuned_vgg_model.train()  # Quay lại chế độ huấn luyện sau khi đánh giá


In [ ]:
plt.plot(val_loss)

In [ ]:
plt.plot(val_acc)
plt.title('Testing Set Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy %')

In [ ]:
models.vgg16

In [ ]:
cifar_fine_tuned_vgg_model = models.vgg11(pretrained='imagenet')  # A
cifar_fine_tuned_vgg_model.classifier[-1].out_features = 10  # A

cifar_fine_tuned_vgg_model.load_state_dict(torch.load('vgg_cifar10.pt', map_location=device))  # B

cifar_finetuned_training_images, shuffled_training_labels, cifar_finetuned_testing_images, shuffled_testing_labels = get_vgg_features(cifar_fine_tuned_vgg_model.features)  # B

print("Fine-tuned VGG11 + LogReg\n=====================")
advanced_grid_search(  # C
    cifar_finetuned_training_images, shuffled_training_labels,
    cifar_finetuned_testing_images, shuffled_testing_labels,
    ml_pipeline, params
)

# A Khởi tạo một mô hình VGG-11 mới
# B Tải các tham số được đào tạo và trích xuất các đặc trưng được tinh chỉnh
# C Chạy tìm kiếm lưới trên các đặc trưng được tinh chỉnh